https://gffpandas.readthedocs.io/en/latest/tutorial.html

In [1]:
import gffpandas.gffpandas as gffpd
import pandas as pd

In [2]:
y22_yps = gffpd.read_gff3('YPS1009_Y22-3_v1_polished.gff')
s288_yps = gffpd.read_gff3('YPS1009_S288C_v5_polished.gff')

In [3]:
y22_unmapped = []
with open('YPS1009_Y22-3_v1_unmapped_features.txt', 'r') as file:
    for line in file:
        y22_unmapped.append(line.strip('\n'))

In [4]:
s288_yps.df['type'].value_counts()

CDS                                   7164
gene                                  6691
mRNA                                  6691
ARS                                    554
noncoding_exon                         512
long_terminal_repeat                   361
tRNA                                   323
tRNA_gene                              323
ARS_consensus_sequence                 199
telomere                                98
transposable_element                    95
transposable_element_gene               95
region                                  82
snoRNA                                  77
snoRNA_gene                             77
LTR_retrotransposon                     53
plus_1_translational_frameshift         47
telomeric_repeat                        42
X_element                               35
X_element_combinatorial_repeat          33
centromere                              30
pseudogene                              25
five_prime_UTR_intron                   24
ncRNA_gene 

In [5]:
def mapping(df):
    id_overlap = []
    dupl = {}
    for ind, row in df.iterrows():
        start = str(row['start'])
        end = str(row['end'])
        
        tmp = str(row['seq_id']) + start + end
        id_overlap.append(tmp)
        if tmp in dupl:
            dupl[tmp] += 1
        else:
            dupl[tmp] = 1
    df['overlap id'] = id_overlap
    filtered_dict = {k:v for (k,v) in dupl.items() if v > 1}
    print('Number of site annotated several time: ' + str(len(filtered_dict)))
    return df, filtered_dict

In [6]:
def drop_overlap(df, filtered_dict):
    df['sequence_ID'] = df['sequence_ID'].astype(float)
    ind = []
    for k in filtered_dict.keys():
        tmp = df[df['overlap id'] == k]
        ind.append(tmp.nsmallest(1,"sequence_ID").index[0])
    df.drop(index=ind, inplace=True)
    return df

In [7]:
# This code just convert the chromosome numbering to integer and remove mitochondrial genes.
def lat_to_arabic(DF):
    Latin_to_arabic = {'I': 1, 'II':2, 'III':3, 'IV':4, 'V':5, 'VI':6, 'VII':7, 'VIII':8, 'IX':9, 'X':10,
                      'XI':11, 'XII':12, 'XIII':13, 'XIV':14, 'XV':15, 'XVI':16, 'Mito': 'mitoch', 'MANY': 7}
    seq_id = DF['seq_id']
    ch = []
    contig = []
    for i in seq_id:
        chrom, cont = i.split('_')
        contig.append(cont)
        chrom = chrom.replace('chr', '')
        if '-' in chrom:
            chrom, e = chrom.split('-')
        ch.append(Latin_to_arabic[chrom])
    DF['Chromosome'] = ch
    DF['Contig'] = contig
    return DF

## Make df by features YPS1009

In [8]:
y22_df = y22_yps.filter_feature_of_type(['gene'])
s288_df = s288_yps.filter_feature_of_type(['gene'])

In [9]:
y22_attr = y22_df.attributes_to_columns()

In [10]:
s288_attr = s288_df.attributes_to_columns()
s288_attr.columns

Index(['seq_id', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase',
       'attributes', 'Alias', 'ID', 'Name', 'Note', 'Ontology_term',
       'copy_num_ID', 'coverage', 'curie', 'dbxref', 'display',
       'extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs'],
      dtype='object')

In [11]:
y22 = y22_attr[['seq_id', 'start', 'end', 'ID', 'Name', 'Dbxref',  'Note', 
       'copy_num_ID', 'coverage', 'extra_copy_number', 'gene', 'low_identity',  'matches_ref_protein', 
                'inframe_stop_codon', 'missing_start_codon', 'missing_stop_codon',
           'partial_mapping', 'sequence_ID', 'valid_ORF', 'valid_ORFs']]

s288 = s288_attr[['seq_id', 'start', 'end', 'ID', 'Name', 'dbxref', 'Alias',  'Note', 'Ontology_term', 
       'copy_num_ID', 'coverage','extra_copy_number', 'gene', 'low_identity', 'orf_classification',
       'partial_mapping', 'sequence_ID', 'valid_ORFs']]

In [12]:
y22 = lat_to_arabic(y22)
s288 = lat_to_arabic(s288)

C:\Users\user\AppData\Local\Temp\ipykernel_17940\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Chromosome'] = ch
C:\Users\user\AppData\Local\Temp\ipykernel_17940\3764198105.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DF['Contig'] = contig
C:\Users\user\AppData\Local\Temp\ipykernel_17940\3764198105.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

# Filter out unvalid ORF

In [15]:
y22 = y22[y22['valid_ORFs'] == '1']
s288 = s288[s288['valid_ORFs'] == '1']

# Look for ORF regions that are only identified in y22

In [18]:
y22, y22_dic = mapping(y22)

Number of site annotated several time: 37


C:\Users\user\AppData\Local\Temp\ipykernel_17940\4120979712.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overlap id'] = id_overlap


In [19]:
# remove dually mapped annotation
y22 = drop_overlap(y22, y22_dic)

C:\Users\user\AppData\Local\Temp\ipykernel_17940\2562817109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sequence_ID'] = df['sequence_ID'].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_17940\2562817109.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index=ind, inplace=True)


In [20]:
s288, s288_dic = mapping(s288)

Number of site annotated several time: 42


C:\Users\user\AppData\Local\Temp\ipykernel_17940\4120979712.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['overlap id'] = id_overlap


In [21]:
s288 = drop_overlap(s288, s288_dic)

C:\Users\user\AppData\Local\Temp\ipykernel_17940\2562817109.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sequence_ID'] = df['sequence_ID'].astype(float)
C:\Users\user\AppData\Local\Temp\ipykernel_17940\2562817109.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(index=ind, inplace=True)


# Now identified rigions not mapped in s288c

In [25]:
only_y22 = list(set(y22['overlap id']) - set(s288['overlap id']))

In [27]:
len(only_y22)

282

In [29]:
only_y22 = y22[y22['overlap id'].isin(only_y22)]

In [30]:
only_y22.to_excel('genes_mapped_only_in_y22.xlsx')

In [ ]:
len(y22[ y22['Dbxref'].isin(list(set(y22['Dbxref']) - set(s288['dbxref'])))])

In [ ]:
y22[ y22['ID'].isin(list(set(y22['ID']) - set(s288['ID'])))]

In [ ]:
no_dbx = y22[y22['Dbxref'].isna()]
no_dbx = no_dbx[no_dbx['valid_ORFs'] == '1']

In [ ]:
no_dbx

In [ ]:
no_dbx, dic = overlap_feat(no_dbx)

In [ ]:
no_dbx = drop_overlap(no_dbx, dic)

In [ ]:
no_dbx

In [ ]:
onlyiny22 = y22[ y22['Dbxref'].isin(list(set(y22['Dbxref']) - set(s288['dbxref'])))]

In [ ]:
onlyiny22

In [ ]:
# export genes not annotated in s288c
onlyiny22.to_excel('genes_annotated_only_with_y22.xlsx')

In [ ]:
ins288_only = s288[s288['dbxref'].isin(list(set(s288['dbxref']) - set(y22['Dbxref'])))]

In [ ]:
ins288_only['valid_ORFs'].value_counts()

Many genes not present in y22 annotations are annotated as invalid in s288c. It's a good sign that those genes may not be needed in wild strains.

In [ ]:
y22['valid_ORFs'].value_counts()
# less unvalid then in s288c annotations.

In [ ]:
y22

In [ ]:
seems like a lot of overlapping features

In [ ]:
# find genes with major overlap (I only round the last digit of start and end position to zero, 
# so not perfect but will cover the biggest overlaps)
id_overlap = []
dupl = {}
for ind, row in y22.iterrows():
    start = str(row['start'])
    start = start.replace(start[len(start) - 1:], '0')
    end = str(row['end'])
    end = end.replace(end[len(end) - 1:], '0')

    tmp = str(row['seq_id']) + start + end
    id_overlap.append(tmp)
    if tmp in dupl:
        dupl[tmp] += 1
    else:
        dupl[tmp] = 1
y22['overlap id'] = id_overlap

In [ ]:
filtered_dict = {k:v for (k,v) in dupl.items() if v > 1}

In [ ]:
len(filtered_dict)
# 64 positions are annotated several time, concern 164 rows

In [ ]:
y22[y22['overlap id'].isin(list(filtered_dict.keys()))]

In [ ]:
# find genes with major overlap (I only round the last digit of start and end position to zero, 
# so not perfect but will cover the biggest overlaps)
id_overlap = []
dupl = {}
for ind, row in s288.iterrows():
    start = str(row['start'])
    start = start.replace(start[len(start) - 1:], '0')
    end = str(row['end'])
    end = end.replace(end[len(end) - 1:], '0')

    tmp = str(row['seq_id']) + start + end
    id_overlap.append(tmp)
    if tmp in dupl:
        dupl[tmp] += 1
    else:
        dupl[tmp] = 1
s288['overlap id'] = id_overlap

In [ ]:
only_mapped_in_y22 = y22[ y22['overlap id'].isin(list(set(y22['overlap id']) - set(s288['overlap id'])))]

In [ ]:
only_mapped_in_y22

In [ ]:
len(set(y22['overlap id']) - set(s288['overlap id']))

In [ ]:
y22[y22['ID'].isin(['YER170W'])]

In [ ]:
missing_stop_v5 = pd.read_csv('missing_stop_v5.txt', sep='\t')

In [ ]:
missing_map_y22 = y22[y22['ID'].isin(list(missing_stop_v5['ID_x']))]

In [ ]:
missing_map_y22['valid_ORFs'].value_counts()

In [ ]:
missing_map_y22[missing_map_y22['valid_ORFs'] == '1']

In [ ]:
missing_merge = pd.merge(missing_stop_v5, missing_map_y22[['ID', 'Name', 'start', 'end', 'valid_ORFs', 'coverage']],  how='left', left_on='ID_x', right_on = 'ID')

In [ ]:
missing_merge.to_excel('missinng_stop_merge_y22.xlsx')

## Same for missing start

In [ ]:
missing_start_v5 = pd.read_csv('missing_start_v5.txt', sep='\t')

In [ ]:
missing_start_map_y22 = y22[y22['ID'].isin(list(missing_start_v5['ID_x']))]

In [ ]:
missing_start_map_y22

In [ ]:
missing_start_map_y22['valid_ORFs'].value_counts()

In [ ]:
missing_start_merge = pd.merge(missing_start_v5, missing_start_map_y22[['ID', 'Name', 'start', 'end', 'valid_ORFs', 'coverage']],  how='left', left_on='ID_x', right_on = 'ID')

In [ ]:
missing_start_merge.to_excel('missing_start_merge_y22.xlsx')

In [31]:
y22

,seq_id,start,end,ID,Name,Dbxref,Note,copy_num_ID,coverage,extra_copy_number,...,inframe_stop_codon,missing_start_codon,missing_stop_codon,partial_mapping,sequence_ID,valid_ORF,valid_ORFs,Chromosome,Contig,overlap id
8,chrII-2_tig00004927,14660,15022,YGR294W_1,WN66_02740,SGD:S000003526,YGR294W~PAU12~Verified,YGR294W_1,1.0,1,...,None,None,None,None,0.997,True,1,2,tig00004927,chrII-2_tig000049271466015022
18,chrIII-2_tig00004935,2752,3702,gene-WN66_00590_1,WN66_00590,SGD:S000000579,YCL074W,gene-WN66_00590_1,0.996,1,...,None,None,None,None,0.986,None,1,3,tig00004935,chrIII-2_tig0000493527523702
28,chrIII_tig00004936,2743,3696,gene-WN66_00590,WN66_00590,SGD:S000000579,YCL074W,gene-WN66_00590_0,1.0,0,...,None,None,None,None,0.990,None,1,3,tig00004936,chrIII_tig0000493627433696
36,chrIII_tig00004936,12617,12742,YCR038W-A_1,WN66_00709,SGD:S000007597,YCR038W-A~Dubious,YCR038W-A_1,1.0,1,...,None,None,None,None,0.992,True,1,3,tig00004936,chrIII_tig000049361261712742
40,chrIII_tig00004936,13765,14292,YCL066W,WN66_00595,SGD:S000000571,YCL066W~HMLALPHA1%2CALPHA1~Verified|silenced_gene,YCL066W_0,1.0,0,...,None,None,None,None,1.000,True,1,3,tig00004936,chrIII_tig000049361376514292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15057,chrX_tig00001098,713888,714784,YNL333W,WN66_05201,SGD:S000005277,YNL333W~SNZ2~Verified,YNL333W_0,1.0,0,...,None,None,None,None,0.995,True,1,10,tig00001098,chrX_tig00001098713888714784
15063,chrX_tig00001098,713888,714784,gene-WN66_03741,WN66_03741,None,potentially,gene-WN66_03741_0,1.0,0,...,None,None,None,None,0.992,None,1,10,tig00001098,chrX_tig00001098713888714784
15067,chrX_tig00001098,715169,715837,YNL334C,WN66_05200,SGD:S000005278,YNL334C~SNO2~Verified,YNL334C_0,1.0,0,...,None,None,None,None,0.979,True,1,10,tig00001098,chrX_tig00001098715169715837
15073,chrX_tig00001098,715169,715837,gene-WN66_03742,WN66_03742,None,potentially,gene-WN66_03742_0,1.0,0,...,None,None,None,None,0.988,None,1,10,tig00001098,chrX_tig00001098715169715837


In [34]:
len(y22[y22['Dbxref'].isna()])

157